In [8]:
import pandas as pd
import numpy as np

from sklearn import preprocessing, model_selection

data_dir = './Dataset/data/'

In [6]:
df = pd.read_csv(data_dir + 'restaurants_laptop_train_with_pos_cleaned.csv')

In [9]:
df.polarity = df.polarity.replace(-1,2)

encoder = preprocessing.LabelEncoder()
df.loc[:, "aspect_tag"] = encoder.fit_transform(df["aspect_tag"])

sentences = df.groupby("num")["text"].apply(list).values
aspect_tags = df.groupby("num")["aspect_tag"].apply(list).values
polarity_tags = df.groupby("num")["polarity"].apply(list).values

polarity_unique_values = df.polarity.unique()

print('num of aspect tags: {}'.format(len(encoder.classes_)))
print('num of polarity tags: {}'.format(len(polarity_unique_values)))

np.where(encoder.classes_ == "AT")[0].item()

num of aspect tags: 2
num of polarity tags: 3


0

In [95]:
idx = 0
print(sentences[idx])
print(aspect_tags[idx])
print(polarity_tags[idx])

['I', 'charge', 'it', 'at', 'night', 'and', 'skip', 'taking', 'the', 'cord', 'with', 'me', 'because', 'of', 'the', 'good', 'battery', 'life']
[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]


In [118]:
n = len(sentences)

full_sentence = []
left_right = []
aspect_term = []
polarity = []


for s in range(n):
    sentence = sentences[s]
    aspect_tag_vector = aspect_tags[s]
    polarity_tag_vector = polarity_tags[s]
    start = 0
    aspect_track = False

    for i in range(len(sentence)):
        # corner case when the aspect term (>2 words) is until last word in the sentence
        if aspect_track == True and aspect_tag_vector[i] == 0 and i == len(sentence)-1:    
            aspect_term.append(sentence[start:i+1])
            left_right.append([sentence[:i+1], []])

        # corner case when the last word is a single aspect
        elif aspect_track == False and aspect_tag_vector[i] == 0 and i == len(sentence)-1:   
            aspect_term.append([sentence[i]])
            polarity.append(polarity_tag_vector[i])
            left_right.append([sentence[:i+1], []])
            full_sentence.append(sentence)

        # reach the end of current aspect term, hence split the sentence and record the instance
        elif aspect_track == True and aspect_tag_vector[i] == 1:
            aspect_track = False 
            #print(start)
            #print(i)
            #print(sentence[start:i])
            aspect_term.append(sentence[start:i])
            right_context = sentence[start:][::-1]
            left_right.append([sentence[:i], right_context])
            
        # the start of aspect term in this sentence
        elif aspect_track == False and aspect_tag_vector[i] == 0:
            start = i
            #print(i)
            #print(sentence[i])
            aspect_track = True
            polarity.append(polarity_tag_vector[i])
            full_sentence.append(sentence)


In [119]:
idx = 2
print(full_sentence[idx])
print(aspect_term[idx])
print(polarity[idx])
print(left_right[idx])

['However', 'the', 'multi', '-', 'touch', 'gestures', 'and', 'large', 'tracking', 'area', 'make', 'having', 'an', 'external', 'mouse', 'unnecessary', 'unless', 'you', "'re", 'gaming']
['multi', '-', 'touch', 'gestures']
1
[['However', 'the', 'multi', '-', 'touch', 'gestures'], ['gaming', "'re", 'you', 'unless', 'unnecessary', 'mouse', 'external', 'an', 'having', 'make', 'area', 'tracking', 'large', 'and', 'gestures', 'touch', '-', 'multi']]


In [120]:
print(len(full_sentence))
print(len(aspect_term))
print(len(left_right))
print(len(polarity))

5731
5731
5731
5731
